In [1]:
import requests
import pandas as pd
import datetime

In [2]:
data = pd.DataFrame(columns=['latitude', 'datacoverage', 'longitude', 'name', 'maxdate', 'id', 'elevationUnit', 'elevation', 'mindate'])

In [3]:
headers = {'token': 'uYBEnqKQRfqjzqnXkuzKBrOYLSKaIMQU'}


r = requests.get('http://www.ncdc.noaa.gov/cdo-web/api/v2/stations?limit=1000&datatypeid=SNWD&extent=48.5518081,12.090589,51.0557185,18.8592361', headers=headers)
for result in r.json()['results']:
    data = data.append(result, ignore_index=True)

In [4]:
stations = {}

for row in data.iterrows():
    if row[1]['name'].endswith('EZ'):
        stations[row[1]['id']] = row[1]['name']

In [5]:
stations

{'GHCND:EZ000011464': 'MILESOVKA, EZ',
 'GHCND:EZM00011406': 'CHEB, EZ',
 'GHCND:EZM00011423': 'PRIMDA, EZ',
 'GHCND:EZM00011487': 'KOCELOVICE, EZ',
 'GHCND:EZM00011518': 'RUZYNE, EZ',
 'GHCND:EZM00011520': 'PRAHA LIBUS, EZ',
 'GHCND:EZM00011603': 'LIBEREC, EZ',
 'GHCND:EZM00011659': 'PRIBYSLAV, EZ',
 'GHCND:EZM00011723': 'TURANY, EZ',
 'GHCND:EZM00011782': 'MOSNOV, EZ',
 'GHCND:EZM00011787': 'LYSA HORA, EZ'}

In [6]:
out = {}

In [7]:
for station in stations:
    time = datetime.datetime.now()
    for i in range(0, 90):
        maxi = time
        mini = time - datetime.timedelta(days=183)
        headers = {'token': 'uYBEnqKQRfqjzqnXkuzKBrOYLSKaIMQU'}
        r = requests.get('http://www.ncdc.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&datatypeid=SNWD&stationid=' + station + '&startdate=' + mini.strftime('%Y-%m-%d') + '&enddate=' + maxi.strftime('%Y-%m-%d') + '&limit=1000', headers=headers)
        try:
            for result in r.json()['results']:
                out[result['date']] = {'snow_depth': result['value'], 'station': stations[station]}
            print(stations[station], maxi.strftime('%Y-%m-%d'), mini.strftime('%Y-%m-%d'))
        except:
            print('!', stations[station], maxi.strftime('%Y-%m-%d'), mini.strftime('%Y-%m-%d'))
        time = mini

LIBEREC, EZ 2015-11-10 2015-05-11
LIBEREC, EZ 2015-05-11 2014-11-09
LIBEREC, EZ 2014-11-09 2014-05-10
LIBEREC, EZ 2014-05-10 2013-11-08
LIBEREC, EZ 2013-11-08 2013-05-09
LIBEREC, EZ 2013-05-09 2012-11-07
LIBEREC, EZ 2012-11-07 2012-05-08
LIBEREC, EZ 2012-05-08 2011-11-07
! LIBEREC, EZ 2011-11-07 2011-05-08
LIBEREC, EZ 2011-05-08 2010-11-06
! LIBEREC, EZ 2010-11-06 2010-05-07
LIBEREC, EZ 2010-05-07 2009-11-05
LIBEREC, EZ 2009-11-05 2009-05-06
LIBEREC, EZ 2009-05-06 2008-11-04
! LIBEREC, EZ 2008-11-04 2008-05-05
LIBEREC, EZ 2008-05-05 2007-11-04
! LIBEREC, EZ 2007-11-04 2007-05-05
LIBEREC, EZ 2007-05-05 2006-11-03
! LIBEREC, EZ 2006-11-03 2006-05-04
LIBEREC, EZ 2006-05-04 2005-11-02
! LIBEREC, EZ 2005-11-02 2005-05-03
LIBEREC, EZ 2005-05-03 2004-11-01
! LIBEREC, EZ 2004-11-01 2004-05-02
LIBEREC, EZ 2004-05-02 2003-11-01
! LIBEREC, EZ 2003-11-01 2003-05-02
LIBEREC, EZ 2003-05-02 2002-10-31
! LIBEREC, EZ 2002-10-31 2002-05-01
LIBEREC, EZ 2002-05-01 2001-10-30
! LIBEREC, EZ 2001-10-30 2001-

In [8]:
outd = pd.DataFrame.from_dict(out, orient='index')

In [11]:
outd.sort(inplace=True)

In [12]:
outd.to_csv('data/snowdepth.csv')